# Descargar datos multiespectrales Sentinel2 y Planet NICFI

In [1]:
# Importar GEE y Iniciar sesion
import ee
import geemap
import os
import geopandas as gpd
import glob

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map(basemap='Esri.WorldImagery')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Importar shapefile

In [4]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Script'

In [5]:
ruta_archivos = r"D:\Descarga"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'D:\\Descarga'

In [6]:
lista = glob.glob("*.shp")
lista

['curvas_de_nivel.shp', 'malla_seleccionada.shp', 'malla_seleccionada3.shp']

In [7]:
# cargar un shp
zona_gdp = gpd.read_file(lista[1])
zona_gdp

,Id,Nombre,geometry
0,0,Parcela_1,"POLYGON ((520237.999 9027505.412, 520237.999 9..."
1,0,Parcela_2,"POLYGON ((520237.999 9032505.412, 520237.999 9..."
2,0,Parcela_3,"POLYGON ((525237.999 9027505.412, 525237.999 9..."
3,0,Parcela_4,"POLYGON ((525237.999 9032505.412, 525237.999 9..."
4,0,Parcela_5,"POLYGON ((530237.999 9027505.412, 530237.999 9..."


## Convertir en Geográfico WGS84

In [8]:
# Hagamos una copia de seguridad de nuestros datos
Zona_geo = zona_gdp.copy()
# Reproject the data
Zona_wg84 = Zona_geo.to_crs(epsg=4326)

In [9]:
Zona_wg84.crs.name

'WGS 84'

## Convertir de Geopandas a GEE

In [10]:
# Crear un featureCollecion mediante GeoJSON
ee_zona_gdp = ee.FeatureCollection(Zona_wg84.__geo_interface__)

In [11]:
geometria = ee_zona_gdp.geometry()

In [12]:
# Visualizar mapa
Map.centerObject(geometria,12)
Map.addLayer(ee_zona_gdp, {"color" : "00FF11"}, "zona_gdp")
Map

Map(bottom=812.0, center=[-8.756990838186095, -74.75689806094516], controls=(WidgetControl(options=['position'…

In [13]:
# Extraer la lista de paises
lista_parcela = ee_zona_gdp.reduceColumns(ee.Reducer.toList(),["Nombre"]).get("list").getInfo()
print(lista_parcela)

['Parcela_1', 'Parcela_2', 'Parcela_3', 'Parcela_4', 'Parcela_5']


In [14]:
Parcela = ee_zona_gdp.filterMetadata('Nombre', 'equals', lista_parcela[0])

In [15]:
os.chdir(r"D:\Descarga")

## Descargar Planet NICFI

In [20]:
# Cargar imagen Planet - NICFI
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')\
            .filter(ee.Filter.date('2023-06-01','2024-04-15'))

In [21]:
# Imprimir la lista de ID Imagenes
ID_planet = nicfi.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_planet)

['planet_medres_normalized_analytic_2023-06_mosaic', 'planet_medres_normalized_analytic_2023-07_mosaic', 'planet_medres_normalized_analytic_2023-08_mosaic', 'planet_medres_normalized_analytic_2023-09_mosaic', 'planet_medres_normalized_analytic_2023-10_mosaic', 'planet_medres_normalized_analytic_2023-11_mosaic', 'planet_medres_normalized_analytic_2023-12_mosaic', 'planet_medres_normalized_analytic_2024-01_mosaic', 'planet_medres_normalized_analytic_2024-02_mosaic', 'planet_medres_normalized_analytic_2024-03_mosaic']


In [22]:
ID_planet[-1]

'planet_medres_normalized_analytic_2024-03_mosaic'

In [23]:
ID_planet[-1][-14:]+"_Planet"

'2024-03_mosaic_Planet'

In [29]:
lista_ID_planet = []

In [30]:
for r in range(len(ID_planet)):
    lista_ID_planet.append(ID_planet[r][-14:])

In [31]:
lista_ID_planet

['2023-06_mosaic',
 '2023-07_mosaic',
 '2023-08_mosaic',
 '2023-09_mosaic',
 '2023-10_mosaic',
 '2023-11_mosaic',
 '2023-12_mosaic',
 '2024-01_mosaic',
 '2024-02_mosaic',
 '2024-03_mosaic']

In [32]:
import pandas as pd

In [33]:
tabla_Planet = pd.DataFrame(lista_ID_planet)

In [35]:
tabla_Planet.columns = ["ID_planet"]

In [36]:
tabla_Planet.head(4)

,ID_planet
0,2023-06_mosaic
1,2023-07_mosaic
2,2023-08_mosaic
3,2023-09_mosaic


In [41]:
# Concatenar string o texto 
out_dir_planet = os.path.normpath((os.getcwd() + '/Planet' ))
out_dir_planet

'D:\\Descarga\\Planet'

In [42]:
# Concatenar string o texto 
out_dir_sentinel2 = os.path.normpath((os.getcwd() + '/Sentinel2' ))
out_dir_sentinel2

'D:\\Descarga\\Sentinel2'

In [45]:
# Crear la carpeta en el directorio
os.makedirs(out_dir_planet)
os.makedirs(out_dir_sentinel2)

In [46]:
os.chdir(out_dir_planet)

In [48]:
tabla_Planet.to_csv(out_dir_planet+"/tabla_ID_Planet.csv")

In [47]:
for i in range(len(lista_parcela)):
    Parcela = ee_zona_gdp.filterMetadata('Nombre', 'equals', lista_parcela[i])
    geometria = Parcela.geometry()
    name_parcela = lista_parcela[i]
    for k in range(len(ID_planet)):
        # Mantiene la convención de nombre de archivo original
        image = ee.Image("projects/planet-nicfi/assets/basemaps/americas" + "/" + ID_planet[k])
        # Reproyectar segun zona de estudio
        img_RS_es = image.reproject(crs="EPSG:32718", scale=4.7)
        # Recortar RS
        img_RS_es_clip = img_RS_es.clip(Parcela)
        # Descarga metodo 2
        geemap.ee_export_image(img_RS_es_clip, 
                               filename=out_dir_planet+"/"+ID_planet[k][-14:]+"_Planet_"+name_parcela+".tif", 
                               region=geometria, 
                               scale=4.77, 
                               file_per_band=False)
        print('Archivo exportado ' + ID_planet[k][-14:]+"_Planet_"+name_parcela+".tif")

Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-06_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-06_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-07_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-07_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-08_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-08_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-09_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-09_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-10_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-10_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-11_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-11_mosaic_Planet_Parcela_1.tif
Generating URL ...
Ple

## Descargar Sentinel2

In [49]:
lista_ID_S2 = []

In [50]:
os.chdir(out_dir_sentinel2)

In [52]:
for i in range(len(lista_parcela)):
    Parcela = ee_zona_gdp.filterMetadata('Nombre', 'equals', lista_parcela[i])
    geometria = Parcela.geometry()
    #Bounds = geometria.bounds()
    #Buffer = Bounds.buffer(300)
    name_parcela = lista_parcela[i]
    # Coleccion de Sentinel-2 SR
    S2_RS = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                    .filterDate('2023-06-01','2024-04-14')\
                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',30)\
                    .filterBounds(Parcela)
    # Imprimir la lista de ID Imagenes
    ID_S2 = S2_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
    # Gardar la lista ID
    for r in range(len(ID_S2)):
        lista_ID_S2.append(ID_S2[r])
    # Proceso de busqueda y descarga
    for k in range(len(ID_S2)):
        # Importar imagen Sentinel-2 RS 
        image = ee.Image("COPERNICUS/S2_SR_HARMONIZED" + "/" + ID_S2[k])
        # Seleccionar iamgenes por bandas
        S2_img_RS = image.select(["B1","B2","B3","B4","B5","B6","B7","B8","B9","B11","B12"])
        # Recortar segun zona estudio
        S2_img_RS_clip = S2_img_RS.clip(Parcela)
        # Descarga Metodo 1 link descarga    
        #link_URL = S2_img_RS_es_clip.getDownloadURL({
        #    'name': ID_S2[k][0:16]+name_parcela,
        #    'bands': ["B1","B2","B3","B4","B5","B6","B7","B8","B9","B11","B12"],
        #    'region': geometria,
        #    'scale': 10,
        #    'filePerBand': False # True = bandas separadas
        #})
        #print('Archivo zip ' + ID_S2[k][0:16]+name_parcela, link_URL)
        
        # Descarga metodo 2
        geemap.ee_export_image(S2_img_RS_clip, 
                               filename=out_dir_sentinel2+"/"+ID_S2[k][0:16]+name_parcela+".tif", 
                               region=geometria, 
                               scale=10, 
                               file_per_band=False)
        print('Archivo exportado ' + ID_S2[k][0:16]+name_parcela+".tif")


Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230619T151711_Parcela_1.tif
Archivo exportado 20230619T151711_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230624T151709_Parcela_1.tif
Archivo exportado 20230624T151709_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230629T151711_Parcela_1.tif
Archivo exportado 20230629T151711_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230704T151709_Parcela_1.tif
Archivo exportado 20230704T151709_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230704T151709_Parcela_1.tif
Archivo exportado 20230704T151709_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230714T151709_Parcela_1.tif
Archivo exportado 20230714T151709_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\

In [53]:
lista_ID = []

In [54]:
for f in range(len(lista_ID_S2)):
    dato = lista_ID_S2[f][0:15]
    lista_ID.append(dato)

In [55]:
len(lista_ID)

210

In [56]:
# Extraer valores únicos como un conjunto
valores_unicos = set(lista_ID)

In [57]:
len(valores_unicos)

20

In [58]:
import pandas as pd

In [59]:
tabla_sentinel2 = pd.DataFrame(valores_unicos)

In [60]:
tabla_sentinel2.columns = ["ID_RS"]

In [61]:
tabla_sentinel2.head(4)

,ID_RS
0,20230922T151709
1,20230917T151711
2,20240325T151721
3,20230808T151711


In [62]:
tabla_sentinel2.to_csv(out_dir_sentinel2+"/"+"tabla_sentinel2_ID_RS.csv")